# Multi-Agent Support
This is an example implementation of tracking events from two separate agents

First let's install the required packages

In [1]:
%pip install -U openai
%pip install -U agentops
%pip install -U groq
%pip install -U python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached groq-0.9.0-py3-none-any.whl.metadata (13 kB)
Using cached groq-0.9.0-py3-none-any.whl (103 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Then import them

In [2]:
import agentops
from agentops import track_agent
from dotenv import load_dotenv
import os
from groq import Groq
import logging

from IPython.display import display, Markdown

Next, we'll grab our API keys. You can use dotenv like below or however else you like to load environment variables

In [3]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"
logging.basicConfig(
    level=logging.DEBUG
)  # this will let us see that calls are assigned to an agent

In [4]:
agentops.init(AGENTOPS_API_KEY, default_tags=["multi-agent-groq-notebook"])
groq_client = Groq(api_key=GROQ_API_KEY)

🖇 AgentOps: The 'tags' parameter is deprecated. Use 'default_tags' instead
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_session HTTP/11" 200 204
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=2f6d54d7-372e-41e5-81df-3d0a31ce23b0
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/howardgil/Desktop/agentops/HowieG/env/lib/python3.12/site-packages/certifi/cacert.pem'


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9


Now lets create a few agents!

In [5]:
@track_agent(name="qa")
class QaAgent:
    def completion(self, prompt: str):
        res = groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "system",
                    "content": "You are a qa engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content


@track_agent(name="engineer")
class EngineerAgent:
    def completion(self, prompt: str):
        res = groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "system",
                    "content": "You are a software engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content

In [6]:
qa = QaAgent()
engineer = EngineerAgent()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443


DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9


Now we have our agents and we tagged them with the `@track_agent` decorator. Any LLM calls that go through this class will now be tagged as agent calls in AgentOps.

Lets use these agents!

In [7]:
generated_func = engineer.completion("python function to test prime number")

DEBUG:groq._base_client:Request options: {'method': 'post', 'url': '/openai/v1/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a software engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'python function to test prime number'}], 'model': 'llama3-70b-8192', 'temperature': 0.5}}
DEBUG:groq._base_client:Sending HTTP Request: POST https://api.groq.com/openai/v1/chat/completions
DEBUG:httpcore.connection:connect_tcp.started host='api.groq.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x124f0bef0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x125456ad0> server_hostname='api.groq.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x12542bbc0>
DEBUG:httpcore.http11:send_reques

In [8]:
display(Markdown("```python\n" + generated_func + "\n```"))

```python
def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    max_divisor = int(n**0.5) + 1
    for d in range(3, max_divisor, 2):
        if n % d == 0:
            return False
    return True
```

In [9]:
generated_test = qa.completion(
    "Write a python unit test that test the following function: \n " + generated_func
)

DEBUG:groq._base_client:Request options: {'method': 'post', 'url': '/openai/v1/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a qa engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python unit test that test the following function: \n def is_prime(n):\n    if n <= 1:\n        return False\n    if n == 2:\n        return True\n    if n % 2 == 0:\n        return False\n    max_divisor = int(n**0.5) + 1\n    for d in range(3, max_divisor, 2):\n        if n % d == 0:\n            return False\n    return True'}], 'model': 'llama3-70b-8192', 'temperature': 0.5}}
DEBUG:groq._base_client:Sending HTTP Request: POST https://api.groq.com/openai/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_requ

In [10]:
display(Markdown("```python\n" + generated_test + "\n```"))

```python
import unittest

def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    max_divisor = int(n**0.5) + 1
    for d in range(3, max_divisor, 2):
        if n % d == 0:
            return False
    return True

class TestIsPrimeFunction(unittest.TestCase):

    def test_is_prime_with_prime_numbers(self):
        self.assertTrue(is_prime(2))
        self.assertTrue(is_prime(3))
        self.assertTrue(is_prime(5))
        self.assertTrue(is_prime(7))
        self.assertTrue(is_prime(11))
        self.assertTrue(is_prime(13))

    def test_is_prime_with_non_prime_numbers(self):
        self.assertFalse(is_prime(0))
        self.assertFalse(is_prime(1))
        self.assertFalse(is_prime(4))
        self.assertFalse(is_prime(6))
        self.assertFalse(is_prime(8))
        self.assertFalse(is_prime(9))

    def test_is_prime_with_negative_numbers(self):
        self.assertFalse(is_prime(-2))
        self.assertFalse(is_prime(-3))
        self.assertFalse(is_prime(-5))
        self.assertFalse(is_prime(-7))
        self.assertFalse(is_prime(-11))
        self.assertFalse(is_prime(-13))

if __name__ == '__main__':
    unittest.main()
```

Perfect! It generated the code as expected, and in the DEBUG logs, you can see that the calls were made by agents named "engineer" and "qa"!

Lets verify one more thing! If we make an LLM call outside of the context of a tracked agent, we want to make sure it gets assigned to the Default Agent.

In [11]:
res = groq_client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are not a tracked agent"},
        {"role": "user", "content": "Say hello"},
    ],
)
res.choices[0].message.content

DEBUG:groq._base_client:Request options: {'method': 'post', 'url': '/openai/v1/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are not a tracked agent'}, {'role': 'user', 'content': 'Say hello'}], 'model': 'llama3-70b-8192'}}
DEBUG:groq._base_client:Sending HTTP Request: POST https://api.groq.com/openai/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 09 Aug 2024 08:21:05 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Cache-Control', b'private, max-age=0

'Hello!'

You'll notice that we didn't log an agent name, so the AgentOps backend will assign it to the Default Agent for the session!